# IMPORT

In [ ]:
try:
    import google.colab # type: ignore
    colab = True
except:
    colab = False

if colab:
    !git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

In [ ]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
 
# Our files
if colab:
    sys.path.append('/content/eai-project/training')
    sys.path.append('/content/eai-project/preprocessing')
    sys.path.append('/content/eai-project/dataset')
    sys.path.append('/content/eai-project/utils')
    sys.path.append('/content/eai-project/models')
    sys.path.append('/content/eai-project/testing')
    from train import train
    from preprocessing import *
    from thermal_dataset import thermal_dataset 
    from utils import *
    from complete_model import complete_model 
    from lstm_only import lstm_only
    from testing import test
    prefix = '/content'
        
else:
    from training.train import train
    from preprocessing.preprocessing import *
    from dataset.thermal_dataset import thermal_dataset
    from utils.csv_utils import *
    from models.complete_model import complete_model
    from models.lstm_only import lstm_only
    from testing.test import test
    prefix = '.'
    
pd.set_option('display.max_columns', None)

In [ ]:
PE_plot = False
not_skip = True
test_flag = True
if os.path.exists(prefix+'/X.npy') and os.path.exists(prefix+'/Y.npy'):
    not_skip = False

**Reproducibility and Device Setting**

In [ ]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# DATASET

In [ ]:
link_zipped_csv = 'https://drive.google.com/file/d/1MssQF4pI_rZqiiDBP4XaLTT1ZaN6ykLm/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)


In [ ]:
if not_skip:

    path = '/content/csv/thermal_drift_features_lab_05_02.csv'


    features_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_02.csv'))
    features_2 = pd.read_csv(os.path.join(prefix, 'csv/thermal_drift_features_lab_05_03.csv'))
    features_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_04.csv'))
    features_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_05.csv'))
    features_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_06.csv'))

    targets_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_02.csv'))
    targets_2 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_03.csv'))
    targets_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_04.csv'))
    targets_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_05.csv'))
    targets_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_06.csv'))
    
    features = pd.concat([features_1,features_2,features_3,features_4,features_5])

    features.drop([
        "name", "tags",
        "2\"Tray1 Vacuum Sensor", "2\"Tray2 Vacuum Sensor", "2\"Tray3 Vacuum Sensor",
        "Avg Oven Temperature", "Chuck Temp [Cdeg]", "Chuck Temp2 [Cdeg]",
        "Chuck1 Vacuum Sensor", "Contrast", "Device State",
        "Dispenser1 Pressure Sensor", "Machine Room Temp", "Main Air", "Main Vacuum",
        "Oven Temperature", "PE_Rx", "PE_Ry", "PE_Rz", "PE_X1", "PE_Y1", "PE_Z1",
        "PUT1 Flow Sensor", "PUT2 Flow Sensor1", "PUT2 Flow Sensor2",
        "PUT2 Flow Sensor3", "PUT2 Flow Sensor4", "PUT2 Flow Sensor5",
        "Photodiode", "Pixel Power", "Preciser1 Vacuum Sensor",
        "Tec FIB1 Holder", "Tec FIB1 Plate", "Tec FIB2 Holder", "Tec FIB2 Plate",
        "Torque11","Torque2","Torque3","Torque4","Torque5","Torque6"
    ], axis=1, inplace=True)

    targets_norm_1= normalize_columns(targets_1, ['X1','Y1','Z1'])
    targets_norm_2= normalize_columns(targets_2, ['X1','Y1','Z1'])
    targets_norm_3= normalize_columns(targets_3, ['X1','Y1','Z1'])
    targets_norm_4= normalize_columns(targets_4, ['X1','Y1','Z1'])
    targets_norm_5= normalize_columns(targets_5, ['X1','Y1','Z1'])

    targets = pd.concat([targets_1,targets_2,targets_3,targets_4,targets_5])
    
    targets.drop(['name', 'tags'], axis=1, inplace=True)


In [ ]:
if not_skip:
    features

In [ ]:
if not_skip:

    targets.head(10)

In [ ]:
if not_skip:

    targets_norm_1['time'] = pd.to_datetime(targets_norm_1['time'], unit='ns')

    # Reshape the DataFrame using melt()
    targets_melted = targets_norm_1.reset_index().melt(id_vars=['time'], value_vars=['X1', 'Y1', 'Z1'], var_name='variable', value_name='value')

    # Drop rows where 'value' is NaN (to keep only the non-null entries)
    targets_melted = targets_melted.dropna(subset=['value'])

    # Plot the data
    plt.figure(figsize=(10, 6))
    for var in targets_melted['variable'].unique():
        # Filter data for each variable and plot
        temp_df = targets_melted[targets_melted['variable'] == var]
        plt.plot(temp_df['time'], temp_df['value'], label=var)

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of X1, Y1, Z1 over Time')
    plt.legend(title='Variable')

    # Show the plot
    plt.show()

    #targets.plot(y='X1',x='time')

In [ ]:
if not_skip:

    # Convert 'time' to datetime (nanoseconds to datetime)
    features['time'] = pd.to_datetime(features_1['time'], unit='ns')

    # Set 'time' as the index
    features.set_index('time', inplace=True)

    # Optionally, you can plot X1, Y1, Z1 directly
    features[['Temp Sensor 1', 'Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4', ]].plot()

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of Temp Sensor 1, Temp Sensor 2, Temp Sensor 3, Temp Sensor 4 over Time')

    # Show the plot
    plt.show()

In [ ]:

if PE_plot and not_skip:
  # Optionally, you can plot X1, Y1, Z1 directly
  features[['PE_X1', 'PE_Y1']].plot()

  # Add labels and title
  plt.xlabel('Time')
  plt.ylabel('Values')
  plt.title('Plot of Temp Sensor 1, Temp Sensor 2, Temp Sensor 3, Temp Sensor 4 over Time')

  # Show the plot
  plt.show()

In [ ]:
if not_skip:

    fixed_targets = transform_dataframe(targets)

In [ ]:
if not_skip:

    fixed_targets

In [ ]:
if not_skip:

    complete_numbers = merge_on_closest_time(fixed_targets.reset_index(), features.reset_index())

In [ ]:
if not_skip:

    complete = complete_numbers.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'Temp1',
        7: 'Temp2',
        8: 'Temp3',
        9: 'Temp4'
        })
    complete.drop(['to_remove', 'time'], axis=1, inplace=True)

In [ ]:
if not_skip:

    complete

In [ ]:
if not_skip:

    complete.astype(float)

    X = complete[['Temp1','Temp2', 'Temp3', 'Temp4']]
    Y = complete[['X1', 'Y1', 'Z1']]

    Y = Y.values.astype(np.float32)
    Y = np.gradient(Y, axis=0)
    X = X.values.astype(np.float32)
    X /= 100

    np.save(prefix+'/X.npy',X)
    np.save(prefix+'/Y.npy',Y)

In [ ]:
if not not_skip:
    X = np.load(prefix+'/X.npy')
    Y = np.load(prefix+'/Y.npy')

print(X)
print(Y)


In [ ]:
def split(listToSplit, splitPercentages):
  length = len(listToSplit)
  splittedList = []
  lastIndex = 0
  for percentageIndex in range(len(splitPercentages)):
    print(lastIndex)
    splittedList.append(listToSplit[lastIndex:lastIndex+int(splitPercentages[percentageIndex]*length)])
    lastIndex += int(splitPercentages[percentageIndex]*length)

  return splittedList

In [ ]:
splitPerc = [0.7,0.2,0.1]
splitted_X = split(X, splitPerc)
splitted_Y = split(Y, splitPerc)

# MODEL

In [ ]:
hyper_parameters = {'num_epochs': 60,
                    'batch_size': 256,
                    'hidden_dim' : 3,
                    'lr': 0.001,
                    'mode': 'auto-weighted', #average, mlp, linear, auto-weighted
                    'extractor_type': 'lstm', #lstm, rnn
                    'ensemble': True,
                    'timesteps': 5
                    }

ensemble_model = {'mlp': {'layer_dim_list': [3,4,3]},  
              'ARIMA': {'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1},
              'linear_regressor': {'in_features': 3, 'out_features': 3}
              }

ensemble_model1 = {'linear_regressor': {'in_features': 3, 'out_features': 3},  
              'ARIMA': {'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1},
              'linear_regressor': {'in_features': 3, 'out_features': 3}
              }

ensemble_model2 = {'mlp': {'layer_dim_list': [3,4,3]},  
              'ARIMA': {'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1},
              'mlp': {'layer_dim_list': [3,4,3]}
              }
#1 ARIMA 2 linear
#1 MLP 2 linear
#
#tutti i voting

In [ ]:
datasetTrain = thermal_dataset((splitted_X[0],splitted_Y[0]), hyper_parameters['timesteps'], device)
datasetTest = thermal_dataset((splitted_X[1],splitted_Y[1]), hyper_parameters['timesteps'], device)
datasetVal = thermal_dataset((splitted_X[2],splitted_Y[2]), hyper_parameters['timesteps'], device)

In [ ]:
# Training, Test and Validation Dataloader initialization
loaderTrain = DataLoader(datasetTrain, shuffle=True, batch_size=hyper_parameters['batch_size'], drop_last=True, collate_fn=pad_collate_fn)
loaderTest = DataLoader(datasetTest, shuffle=True, batch_size=hyper_parameters['batch_size'], drop_last=True, collate_fn=pad_collate_fn)
loaderVal = DataLoader(datasetVal, shuffle=True, batch_size=hyper_parameters['batch_size'], drop_last=True, collate_fn=pad_collate_fn)

# Model Initialization (True if you want to use the Ensemble model, False in you want to use a single LSTM model)
if hyper_parameters['ensemble'] == False:
    model = lstm_only(hidden_dim=hyper_parameters['hidden_dim'], timesteps=hyper_parameters['timesteps'] ,input_dim=4, output_dim=3).to(device)
elif hyper_parameters['ensemble'] == True:  
    model = complete_model(hidden_dim=hyper_parameters['hidden_dim'], input_dim=4, model_dict=ensemble_model, device=device, mode=hyper_parameters['mode'], extractor_type=hyper_parameters['extractor_type']).to(device)

# Definition of the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=hyper_parameters['lr'])
loss_fn = nn.MSELoss()
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [ ]:
if not test_flag:
    train(
        num_epochs=hyper_parameters['num_epochs'],
        loss_fn=loss_fn,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        training_dataloader=loaderTrain,
        validation_dataloader=loaderVal,
        hyperparams=hyper_parameters,
        model_dict = ensemble_model,
        complete=hyper_parameters['ensemble']
    )


# Test

In [ ]:
# Need to specify a model.pt file to load! In the directory results that you want to test
result_path = '/results/ensemble_batch_256' 
model_path = prefix + result_path +'/model.pt'
if test_flag:
    test_metrics, test_loss_avg, total_inference_time, inference_time_avg = test(
        model=model,
        model_path=model_path,
        test_dataloader=loaderTest,
        loss_fn=loss_fn,
        complete=hyper_parameters['ensemble']
    )
print("Test metrics:")
print(f"rmse: {test_metrics['rmse']} , mae: {test_metrics['mae']}, r2: {test_metrics['r2']}")
print("Others:")
print(f"Average Loss: {test_loss_avg}, Total Inference Time (All Dataset): {total_inference_time}, Average Inference Time: {inference_time_avg}")


In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

# Compute training loss as the square of rmse for each element because loss = mse
# Both for training and validation
training_loss = [rmse**2 for rmse in train_metrics['rmse']] 
valid_loss = [rmse**2 for rmse in valid_metrics['rmse']] 

plt.plot(epochs, training_loss, label='Training', color='b')
plt.plot(epochs, valid_loss, label='Validation', color='r')
plt.title('Loss (RMSE^2)')
plt.xlabel('Epochs')
plt.ylabel('Loss value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path+'/loss')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

plt.plot(epochs, train_metrics['mae'], label='Training', color='b')
plt.plot(epochs, valid_metrics['mae'], label='Validation', color='r')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path + '/mae')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

plt.plot(epochs, train_metrics['r2'], label='Training', color='b')
plt.plot(epochs, valid_metrics['r2'], label='Validation', color='r')
plt.title('R2 (Coefficient of determination)')
plt.xlabel('Epochs')
plt.ylabel('R2 value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()